In [1]:
import pandas as pd
from umap import UMAP

# Load CSV with latin-1 encoding and semicolon separator
df = pd.read_csv("downloads/lic_2020-1.csv", encoding="latin-1", sep=";")

# Identify numeric columns (excluding CodigoExterno)
# Convert columns to numeric where possible, handling comma decimal separators
numeric_columns = []
for col in df.columns:
    if col in (
        "CodigoExterno",
        "Codigo",
        "CodigoEstado",
        "EstadoEtapas",
        "CodigoUnidad",
        "Informada",
        "EsBaseTipo",
        "ValorTiempoRenovacion",
        "EsRenovable",
        "Codigoitem",
        "CodigoProductoONU",
        "CodigoSucursalProveedor",
        "Correlativo",
    ):
        continue
    # Try to convert to numeric, handling comma decimal separators
    # Replace comma with dot for decimal separator
    test_series = df[col].astype(str).str.replace(",", ".", regex=False)
    numeric_series = pd.to_numeric(test_series, errors="coerce")
    # Check if column is numeric (has valid numeric values and not all NaN)
    if numeric_series.notna().any():
        # Check if the column is actually numeric (most values are numeric)
        non_null_count = numeric_series.notna().sum()
        total_count = len(numeric_series)
        # Consider it numeric if at least 50% of values are numeric
        if non_null_count / total_count >= 0.5:
            numeric_columns.append(col)

# Prepare numeric data for UMAP
numeric_data = df[numeric_columns].copy()
# Convert to numeric, handling comma decimal separators
for col in numeric_columns:
    # Replace comma with dot for decimal separator, then convert to numeric
    numeric_data[col] = numeric_data[col].astype(str).str.replace(",", ".", regex=False)
    numeric_data[col] = pd.to_numeric(numeric_data[col], errors="coerce")
# Fill missing values with 0 (or could use median/mean)
numeric_data = numeric_data.fillna(0)
# Ensure all values are float (not object/string)
numeric_data = numeric_data.astype(float).drop_duplicates()

/home/victor/Documentos/Coding/hack-25/notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2336067/1569871205.py:5: DtypeWarning: Columns (27,52,63,64,87,97,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("downloads/lic_2020-1.csv", encoding="latin-1", sep=";")


In [35]:
col_uniques = []
for column in numeric_data.columns:
    col_uniques.append((column, numeric_data[column].nunique()))

sorted(col_uniques, key=lambda x: x[1], reverse=True)

[('Valor Total Ofertado', 62251),
 ('MontoUnitarioOferta', 43954),
 ('MontoLineaAdjudica', 19674),
 ('CodigoProveedor', 11895),
 ('Monto Estimado Adjudicado', 6857),
 ('MontoEstimado', 3463),
 ('NumeroAprobacion', 2047),
 ('CantidadAdjudicada', 1306),
 ('Cantidad Ofertada', 1183),
 ('Cantidad', 1181),
 ('CodigoOrganismo', 724),
 ('CantidadReclamos', 407),
 ('TiempoDuracionContrato', 89),
 ('NumeroOferentes', 62),
 ('FechaTiempoEvaluacion', 37),
 ('CodigoEstadoLicitacion', 10),
 ('FechasUsuario', 8),
 ('TipoAprobacion', 7),
 ('UnidadTiempoDuracionContrato', 5),
 ('Estimacion', 4),
 ('TipoPago', 4),
 ('EstadoCS', 3),
 ('Contrato', 3),
 ('UnidadTiempoContratoLicitacion', 3),
 ('CodigoTipo', 2),
 ('TipoConvocatoria', 2),
 ('Etapas', 2),
 ('TomaRazon', 2),
 ('EstadoPublicidadOfertas', 2),
 ('Obras', 2),
 ('VisibilidadMonto', 2),
 ('SubContratacion', 2),
 ('ExtensionPlazo', 2)]

In [4]:
numeric_data_only_awards = numeric_data[numeric_data["CantidadAdjudicada"] > 0].drop_duplicates()
df_awards = df.iloc[numeric_data_only_awards.index].copy()

In [8]:
def format_text_for_embedding(row):
    return "\n\n".join(
        [
            (row["Nombre"] if not pd.isna(row["Nombre"]) else ""),
            (row["Descripcion"] if not pd.isna(row["Descripcion"]) else ""),
            (
                row["Nombre producto genrico"]
                if not pd.isna(row["Nombre producto genrico"])
                else ""
            ),
            (
                row["Descripcion linea Adquisicion"]
                if not pd.isna(row["Descripcion linea Adquisicion"])
                else ""
            ),
            (
                row["DescripcionProveedor"]
                if not pd.isna(row["DescripcionProveedor"])
                else ""
            ),
        ]
    )


df_awards.loc[:, "compiled_text"] = df_awards.apply(format_text_for_embedding, axis=1)
df_awards.loc[:, "supplier_rut"] = df_awards["RutProveedor"].map(
    lambda x: x.split("-")[0].replace(".", "")
)

In [ ]:
# Compute text embeddings using SentenceTransformer
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get compiled_text for awards data (matching the indices)
texts = df_awards["compiled_text"].fillna("").tolist()

# Compute embeddings efficiently in batches
print(f"Computing embeddings for {len(texts)} texts...")
text_embeddings = model.encode(
    texts, batch_size=128, show_progress_bar=True, convert_to_numpy=True
)

print(f"Text embeddings shape: {text_embeddings.shape}")
print(f"Embedding dimension: {text_embeddings.shape[1]}")


Computing embeddings for 42874 texts...


Batches: 100%|██████████| 335/335 [00:11<00:00, 28.37it/s]


Text embeddings shape: (42874, 384)
Embedding dimension: 384


In [7]:
# Concatenate text embeddings with numeric columns
# Ensure numeric_data_only_awards is aligned with text_embeddings
numeric_array = numeric_data_only_awards.values.astype(np.float32)

# Check for and handle infinite values
if np.any(np.isinf(numeric_array)):
    print("Warning: Found infinite values, replacing with NaN")
    numeric_array = np.where(np.isinf(numeric_array), np.nan, numeric_array)

# Replace any remaining NaN with 0
numeric_array = np.nan_to_num(numeric_array, nan=0.0, posinf=0.0, neginf=0.0)

# Concatenate text embeddings (text_embeddings) with numeric columns (numeric_array)
# Result: (n_samples, embedding_dim + n_numeric_features)
combined_features = np.concatenate([text_embeddings, numeric_array], axis=1)

print(f"Combined features shape: {combined_features.shape}")
print(f"  - Text embedding dimension: {text_embeddings.shape[1]}")
print(f"  - Numeric columns dimension: {numeric_array.shape[1]}")
print(f"  - Total dimension: {combined_features.shape[1]}")

# Check for duplicate rows (can cause issues with nearest neighbor search)
# Add tiny random noise to duplicate rows to make them unique
unique_rows, unique_indices, inverse_indices = np.unique(
    combined_features, axis=0, return_index=True, return_inverse=True
)
if len(unique_rows) < len(combined_features):
    print(
        f"Warning: Found {len(combined_features) - len(unique_rows)} duplicate rows, adding small noise"
    )
    # Add very small random noise to make duplicates unique
    np.random.seed(42)
    noise = np.random.normal(0, 1e-8, combined_features.shape).astype(np.float32)
    combined_features = combined_features + noise

# Apply Dimensionality Reduction to the combined features
reducer = UMAP(n_components=2, random_state=42)
umap_embedding = reducer.fit_transform(combined_features)

Combined features shape: (42874, 417)
  - Text embedding dimension: 384
  - Numeric columns dimension: 33
  - Total dimension: 417


/home/victor/Documentos/Coding/hack-25/notebooks/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/home/victor/Documentos/Coding/hack-25/notebooks/.venv/lib/python3.12/site-packages/sklearn/manifold/_spectral_embedding.py:328: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [9]:
df_act = pd.read_csv("downloads/PUB_EMPRESAS_PJ_2020_A_2024.txt", sep="\t")
# Año comercial	RUT	DV	Razón social	Tramo según ventas
# Número de trabajadores dependie	Fecha inicio de actividades vige	Fecha término de giro
# Fecha primera inscripción de ac	Tipo término de giro
# Tipo de contribuyente	Subtipo de contribuyente	Tramo capital propio positivo	Tramo capital propio negativo
# Rubro económico	Subrubro económico	Actividad económica
# Región	Provincia	Comuna
# R_PRESUNTA	OTROS_REGIMENES
df_act.columns = [
    "fiscal_year",
    "rut",
    "dv",
    "company_name",
    "sales_bracket",
    "num_employees",
    "current_activity_start_date",
    "activity_end_date",
    "first_registration_date",
    "activity_end_type",
    "contributor_type",
    "contributor_subtype",
    "positive_equity_bracket",
    "negative_equity_bracket",
    "economic_sector",
    "economic_subsector",
    "economic_activity",
    "region",
    "province",
    "commune",
    "presumed_income",
    "other_regimes",
]
df_act.first_registration_date = pd.to_datetime(
    df_act.first_registration_date, errors="coerce"
)
df_act.current_activity_start_date = pd.to_datetime(
    df_act.current_activity_start_date, errors="coerce"
)
df_act.activity_end_date = pd.to_datetime(df_act.activity_end_date, errors="coerce")

rut_to_registration_date = {str(d["rut"]): d["first_registration_date"] for d in df_act[["rut", "first_registration_date"]].to_dict(orient="records")}
df_awards.loc[:, "first_activity_date"] = df_awards["supplier_rut"].map(rut_to_registration_date.get)

/tmp/ipykernel_2336067/296279619.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_act = pd.read_csv("downloads/PUB_EMPRESAS_PJ_2020_A_2024.txt", sep="\t")


In [10]:
# Create final DataFrame with CodigoExterno, numeric columns, and UMAP x, y
result_df = pd.DataFrame()
result_df["CodigoExterno"] = df_awards["CodigoExterno"]
result_df["tender_name"] = df_awards["Nombre"]
result_df["supplier_name"] = df_awards["RazonSocialProveedor"]
result_df["supplier_rut"] = df_awards["supplier_rut"]
result_df["first_activity_date"] = pd.to_datetime(df_awards["first_activity_date"])

for col in df_awards.columns:
    if col.startswith("Fecha"):
        result_df[col] = pd.to_datetime(df_awards[col])
# Add all numeric columns
for col in numeric_columns:
    result_df[col] = numeric_data_only_awards[col]
# Add UMAP x and y columns
result_df["x"] = umap_embedding[:, 0]
result_df["y"] = umap_embedding[:, 1]
result_df.reset_index(drop=True, inplace=True)
# Display result
print(f"Shape: {result_df.shape}")
print(f"Numeric columns found: {len(numeric_columns)}")
print(f"Columns: {list(result_df.columns[:5])}... (showing first 5)")
result_df.head()

Shape: (42874, 56)
Numeric columns found: 33
Columns: ['CodigoExterno', 'tender_name', 'supplier_name', 'supplier_rut', 'first_activity_date']... (showing first 5)


,CodigoExterno,tender_name,supplier_name,supplier_rut,first_activity_date,FechaCreacion,FechaCierre,FechaInicio,FechaFinal,FechaPubRespuestas,...,Cantidad,CodigoProveedor,Monto Estimado Adjudicado,Cantidad Ofertada,MontoUnitarioOferta,Valor Total Ofertado,CantidadAdjudicada,MontoLineaAdjudica,x,y
0,812030-5-LQ19,SUMINISTRO DE INMUNOGLOBULINA,GRIFOLS CHILE S A,96582310,1993-01-01,2019-01-18,2020-01-31,2020-01-21,2020-01-22,2020-01-24,...,10.0,54808.0,223973946.0,10.0,159200.0,1592000.0,10.0,1592000.0,14.278939,17.310768
1,812030-5-LQ19,SUMINISTRO DE INMUNOGLOBULINA,GRIFOLS CHILE S A,96582310,1993-01-01,2019-01-18,2020-01-31,2020-01-21,2020-01-22,2020-01-24,...,60.0,54808.0,223973946.0,60.0,322010.0,19320600.0,60.0,19320600.0,-4.153696,-7.208119
2,2258-58-LE19,Licitacion de BACK UP para laboratorio,BIOMERIEUX CHILE SPA,96659920,1993-01-01,2019-03-11,2020-01-20,2020-01-10,2020-01-15,2020-01-17,...,1.0,33139.0,20954265.0,1.0,1750.0,1750.0,5000.0,8750000.0,-1.487097,-0.097432
3,2258-58-LE19,Licitacion de BACK UP para laboratorio,QUORUX CHILE SPA,76131142,2011-01-26,2019-03-11,2020-01-20,2020-01-10,2020-01-15,2020-01-17,...,1.0,1333599.0,20954265.0,1.0,660.0,660.0,2400.0,1584000.0,-1.616803,0.211644
4,2258-58-LE19,Licitacion de BACK UP para laboratorio,QUORUX CHILE SPA,76131142,2011-01-26,2019-03-11,2020-01-20,2020-01-10,2020-01-15,2020-01-17,...,1.0,1333599.0,20954265.0,1.0,1100.0,1100.0,2400.0,2640000.0,-1.575541,0.177008


In [11]:
result_df.to_parquet('downloads/lic_2020-1_umap.parquet')